<a href="https://colab.research.google.com/github/nuriamontala/PYT-SBI/blob/main/model_%26_prediction_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import torch
import torch.nn as nx
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_add_pool
from torch_geometric.data import Data
import graphein.protein as gp
from graphein.protein.config import ProteinGraphConfig
from Bio import PDB
import os
from torch_geometric.data import Data, Dataset
import pandas as pd
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.graphs import construct_graph
from graphein.protein.edges.distance import (add_peptide_bonds,
                                             add_hydrogen_bond_interactions,
                                             add_disulfide_interactions,
                                             add_ionic_interactions,
                                             add_aromatic_interactions,
                                             add_aromatic_sulphur_interactions,
                                             add_cation_pi_interactions,
                                             add_delaunay_triangulation)

# Edge construction functions
new_edge_funcs = {"edge_construction_functions": [
    add_peptide_bonds,
    add_aromatic_interactions,
    add_hydrogen_bond_interactions,
    add_disulfide_interactions,
    add_ionic_interactions,
    add_aromatic_sulphur_interactions,
    add_cation_pi_interactions,
    add_delaunay_triangulation
]}


In [42]:
# ✅ Load Residue Features from CSV
def load_residue_features(csv_path):
    """Loads residue features and binding site labels from CSV."""
    df = pd.read_csv(csv_path)

    # Convert first column (Residue ID) into tuples (residue_number, chain)
    df.iloc[:, 0] = df.iloc[:, 0].apply(eval)  # Convert string "(1, 'L')" → tuple (1, 'L')

    residue_ids = df.iloc[:, 0]  # Residue ID as tuple
    features = df.iloc[:, 1:-1].values  # Feature columns (excluding label)
    labels = df.iloc[:, -1].values  # Last column = binding site labels

    # Create lookup dictionaries
    features_dict = {res_id: feat for res_id, feat in zip(residue_ids, features)}
    labels_dict = {res_id: label for res_id, label in zip(residue_ids, labels)}

    return features_dict, labels_dict, features.shape[1]  # Return num_features

# ✅ Convert NetworkX Graph → PyTorch Geometric Graph
def networkx_to_pyg(G_nx, features_dict, labels_dict, num_features):
    """Converts a NetworkX protein graph to a PyTorch Geometric Data object with features & labels."""
    node_map = {}  # Maps node (residue, chain) to index
    reverse_map = {}  # Reverse lookup: PyG index → (residue_number, chain)
    node_features = []
    y = []

    for i, (node, attr) in enumerate(G_nx.nodes(data=True)):
        res_id = (attr.get("residue_number"), attr.get("chain_id"))  # Standardized format

        node_map[node] = i  # Assign PyG-compatible node index
        reverse_map[i] = res_id  # Store mapping back to residue identifier

        # Retrieve features (default to zero vector if missing)
        features = features_dict.get(res_id, [0] * num_features)
        node_features.append(features)

        # Retrieve binding site labels (default = non-binding)
        y.append(labels_dict.get(res_id, 0))

    x = torch.tensor(node_features, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.long)

    # Extract edges
    edges = []
    for u, v in G_nx.edges():
        try:
            edges.append((node_map[u], node_map[v]))  # Use fixed node IDs
        except KeyError:
            print(f"⚠️ Skipping edge ({u}, {v}) due to missing node mapping!")

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

    return Data(x=x, edge_index=edge_index, y=y), reverse_map

# ✅ Process All PDB + CSV Files in a Folder
def process_protein_graphs(folder_path):
    """Processes all PDB files in a given folder & returns a ProteinGraphDataset."""
    protein_graphs = []

    # Get all PDB files in folder
    pdb_files = [f for f in os.listdir(folder_path) if f.endswith(".pdb")]

    for pdb_file in pdb_files:
        pdb_path = os.path.join(folder_path, pdb_file)
        csv_path = os.path.join(folder_path, pdb_file.replace(".pdb", ".csv"))

        if not os.path.exists(csv_path):
            print(f"⚠️ Warning: No CSV found for {pdb_file}. Skipping!")
            continue  # Skip this PDB if no matching CSV

        print(f"📌 Processing {pdb_file}...")

        # Load features & labels
        features_dict, labels_dict, num_features = load_residue_features(csv_path)

        # Construct NetworkX graph
        config = ProteinGraphConfig(**new_edge_funcs)
        G_nx = construct_graph(config=config, path=pdb_path)

        # Convert to PyG Data object
        protein_graph_data, reverse_map = networkx_to_pyg(G_nx, features_dict, labels_dict, num_features)

        # Store the graph
        protein_graphs.append(protein_graph_data)

        print(f"✅ Processed {pdb_file} ({len(G_nx.nodes())} nodes, {len(G_nx.edges())} edges)")

    return ProteinGraphDataset(protein_graphs)

# ✅ Define PyTorch Geometric Dataset
class ProteinGraphDataset(Dataset):
    """Custom PyTorch Geometric Dataset to handle protein graphs."""
    def __init__(self, protein_graphs):
        super().__init__()
        self.protein_graphs = protein_graphs  # List of Data objects

    def len(self):
        return len(self.protein_graphs)

    def get(self, idx):
        return self.protein_graphs[idx]

# ✅ Define Virtual Node GAT Model
class GATVirtualNode(nn.Module):
    def __init__(self, in_features, hidden_dim, num_heads, num_classes, dropout=0.2):
        super(GATVirtualNode, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads

        # Virtual Node (Global Graph Representation)
        self.virtual_node_embedding = nn.Parameter(torch.zeros(1, hidden_dim))

        # Graph Attention Layers
        self.gat1 = GATConv(in_features, hidden_dim // num_heads, heads=num_heads)
        self.gat2 = GATConv(hidden_dim, hidden_dim // num_heads, heads=num_heads)

        # MLP Classifier for **node classification**
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, num_classes)  # Output for **each node**
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply first GAT layer
        x = self.gat1(x, edge_index)
        x = torch.relu(x)
        x = self.dropout(x)

        # Apply second GAT layer
        x = self.gat2(x, edge_index)
        x = torch.relu(x)
        x = self.dropout(x)

        # Virtual Node Aggregation
        virtual_node = self.virtual_node_embedding.expand(x.size(0), -1)
        x = x + virtual_node

        # **Return node-level predictions**
        return self.mlp(x)  # Output has same shape as `data.y`


# ✅ Load & Prepare Dataset
folder_path = "testingfiles"  # CHANGE THIS!
protein_graph_dataset = process_protein_graphs(folder_path)

# ✅ Split dataset into train/test
train_size = int(0.8 * len(protein_graph_dataset))
test_size = len(protein_graph_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(protein_graph_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# ✅ Define Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GATVirtualNode(in_features=protein_graph_dataset[0].x.shape[1], hidden_dim=128, num_heads=4, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()


model_save_path = "trained_gat_model.pth"  # Change if needed

# ✅ Training Loop (with saving)
num_epochs = 30
best_acc = 0.0  # Track best accuracy to save the best model

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    test_acc = evaluate(model, test_loader, device)

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}")

    # ✅ Save best model
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), model_save_path)
        print(f"✅ Model saved at epoch {epoch+1} with accuracy {test_acc:.4f}!")

print("✅ Training Completed!")



Output()

📌 Processing 2asu.pdb...


Output()

✅ Processed 2asu.pdb (229 nodes, 1601 edges)
📌 Processing 6h5w.pdb...


Output()

...

Output()

✅ Processed 5f1u.pdb (1184 nodes, 8827 edges)
📌 Processing 1awh.pdb...


✅ Processed 1awh.pdb (590 nodes, 4414 edges)
Epoch 1/30 - Loss: 0.6386, Test Accuracy: 0.9519
✅ Model saved at epoch 1 with accuracy 0.9519!
Epoch 2/30 - Loss: 0.6146, Test Accuracy: 0.9519


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 3/30 - Loss: 0.5901, Test Accuracy: 0.9519
Epoch 4/30 - Loss: 0.5627, Test Accuracy: 0.9519
Epoch 5/30 - Loss: 0.5343, Test Accuracy: 0.9519
Epoch 6/30 - Loss: 0.5041, Test Accuracy: 0.9519
Epoch 7/30 - Loss: 0.4742, Test Accuracy: 0.9519
Epoch 8/30 - Loss: 0.4449, Test Accuracy: 0.9519
Epoch 9/30 - Loss: 0.4207, Test Accuracy: 0.9519
Epoch 10/30 - Loss: 0.4040, Test Accuracy: 0.9519
Epoch 11/30 - Loss: 0.3936, Test Accuracy: 0.9519
Epoch 12/30 - Loss: 0.3971, Test Accuracy: 0.9519
Epoch 13/30 - Loss: 0.4046, Test Accuracy: 0.9519
Epoch 14/30 - Loss: 0.4082, Test Accuracy: 0.9519
Epoch 15/30 - Loss: 0.4146, Test Accuracy: 0.9519
Epoch 16/30 - Loss: 0.4143, Test Accuracy: 0.9519
Epoch 17/30 - Loss: 0.4076, Test Accuracy: 0.9519
Epoch 18/30 - Loss: 0.4033, Test Accuracy: 0.9519
Epoch 19/30 - Loss: 0.3951, Test Accuracy: 0.9519
Epoch 20/30 - Loss: 0.3868, Test Accuracy: 0.9519
Epoch 21/30 - Loss: 0.3810, Test Accuracy: 0.9519
Epoch 22/30 - Loss: 0.3764, Test Accuracy: 0.9519
Epoch 2

In [ ]:
import torch
import os
import pandas as pd
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.graphs import construct_graph
from torch_geometric.data import Data
import torch.nn.functional as F  # Import softmax

# ✅ Load Residue Features from CSV (without labels)
def load_residue_features(csv_path):
    """Loads residue features (without labels) from CSV."""
    df = pd.read_csv(csv_path)
    df.iloc[:, 0] = df.iloc[:, 0].apply(eval)  # Convert "(1, 'L')" → (1, 'L')

    residue_ids = df.iloc[:, 0]  # Residue ID as tuple
    features = df.iloc[:, 1:].values  # Feature columns (WITHOUT label)

    # Create lookup dictionary
    features_dict = {res_id: feat for res_id, feat in zip(residue_ids, features)}

    return features_dict, features.shape[1]  # Return features & number of features

# ✅ Convert NetworkX Graph → PyTorch Geometric Graph
def networkx_to_pyg(G_nx, features_dict, num_features):
    """Converts a NetworkX protein graph to a PyTorch Geometric Data object."""
    node_map = {}
    reverse_map = {}
    node_features = []

    for i, (node, attr) in enumerate(G_nx.nodes(data=True)):
        res_id = (attr.get("residue_number"), attr.get("chain_id"))
        node_map[node] = i
        reverse_map[i] = res_id

        features = features_dict.get(res_id, [0] * num_features)  # Use zero vector if missing
        node_features.append(features)

    x = torch.tensor(node_features, dtype=torch.float)

    edges = [(node_map[u], node_map[v]) for u, v in G_nx.edges() if u in node_map and v in node_map]
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

    return Data(x=x, edge_index=edge_index), reverse_map

# ✅ Function to Predict Binding Sites with Probabilities
def predict_binding_sites(model, pdb_path, csv_path, device):
    """Predicts binding sites for a single PDB file using a trained model and returns probabilities."""
    model.eval()

    if not os.path.exists(csv_path):
        print(f"⚠️ No CSV found for {pdb_path}. Skipping!")
        return None

    print(f"📌 Processing {pdb_path}...")

    # Load features
    features_dict, num_features = load_residue_features(csv_path)

    # Construct graph
    config = ProteinGraphConfig(**new_edge_funcs)
    G_nx = construct_graph(config=config, path=pdb_path)

    # Convert to PyG format
    protein_graph_data, reverse_map = networkx_to_pyg(G_nx, features_dict, num_features)

    # Move to device
    protein_graph_data = protein_graph_data.to(device)

    # Make prediction
    with torch.no_grad():
        out = model(protein_graph_data)  # Shape: (num_nodes, num_classes)
        probabilities = F.softmax(out, dim=1)[:, 1]  # Probability of class 1 (binding site)

    # Map probabilities to residue IDs
    binding_site_probabilities = {
        reverse_map[i]: float(probabilities[i].item()) for i in range(len(probabilities))
    }

    return binding_site_probabilities

# ✅ Load Trained Model
model_path = "trained_gat_model.pth"  # CHANGE THIS to your model path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load dynamic in_features based on dataset
pdb_file = "6afj.pdb"  # CHANGE THIS
csv_file = "6afj.csv"  # CHANGE THIS

_, num_features = load_residue_features(csv_file)  # Get correct input size

# ✅ Initialize Model (using correct feature size)
model = GATVirtualNode(
    in_features=num_features, hidden_dim=128, num_heads=4, num_classes=2
).to(device)

model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# ✅ Inference on a New PDB + CSV (with probabilities)
binding_site_probs = predict_binding_sites(model, pdb_file, csv_file, device)

# ✅ Print Results
print("\n🔹 Binding Site Probability Predictions 🔹")
for residue, prob in binding_site_probs.items():
    print(f"Residue {residue}: Probability of Binding Site = {prob:.4f}")


Output()

<ipython-input-21-997fc6dfe1bf>:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


📌 Processing 6afj.pdb...



🔹 Binding Site Probability Predictions 🔹
Residue (2, 'A'): Probability of Binding Site = 0.1256
Residue (3, 'A'): Probability of Binding Site = 0.1347
Residue (4, 'A'): Probability of Binding Site = 0.1403
Residue (5, 'A'): Probability of Binding Site = 0.1444
Residue (6, 'A'): Probability of Binding Site = 0.1715
Residue (7, 'A'): Probability of Binding Site = 0.1857
Residue (8, 'A'): Probability of Binding Site = 0.1820
Residue (9, 'A'): Probability of Binding Site = 0.1683
Residue (10, 'A'): Probability of Binding Site = 0.1563
Residue (11, 'A'): Probability of Binding Site = 0.1485
Residue (12, 'A'): Probability of Binding Site = 0.1590
Residue (13, 'A'): Probability of Binding Site = 0.1920
Residue (14, 'A'): Probability of Binding Site = 0.1715
Residue (15, 'A'): Probability of Binding Site = 0.1783
Residue (16, 'A'): Probability of Binding Site = 0.1728
Residue (17, 'A'): Probability of Binding Site = 0.1939
Residue (18, 'A'): Probability of Binding Site = 0.1953
Residue (19, '